In [1]:
! pip install transformers
! pip install torch torchvision torchaudio

In [38]:
# MODEL = 'fine_tuned/NER/ACD_1epoch'
# MODEL = 'fine_tuned/NER/ACD_5epochs'
MODEL = 'fine_tuned/NER/ACD_10epochs'




MODEL_PATH = '../../models/word_embeddings/'

MODEL_PATH += MODEL

In [39]:
import os, json
from transformers import AutoConfig

config = AutoConfig.from_pretrained(MODEL_PATH)

labels = []
labels = [value for k, value in config.id2label.items()]

print(config.id2label)

{0: 'B-Anatomy', 1: 'B-Chemical', 2: 'B-Disease', 3: 'I-Anatomy', 4: 'I-Chemical', 5: 'I-Disease', 6: 'O'}


In [41]:
from transformers import BertForTokenClassification, BertTokenizer

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH, do_lower_case=False)

model = BertForTokenClassification.from_pretrained(
    MODEL_PATH,
    num_labels=len(labels),
    output_attentions = False,
    output_hidden_states = False
)

In [43]:
import torch
import numpy as np

# test_sentence = 'Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a “heart attack” at my age and I am very worried about it.'
# test_sentence = " Her eye is green. A 58-year-old African-American woman presents to the ER with episodic pressing/burning anterior chest pain that began two days earlier for the first time in her life. The pain started while she was walking, radiates to the eyes, and is accompanied by nausea, diaphoresis and mild dyspnea, but is not increased on inspiration. The latest episode of pain ended half an hour prior to her arrival. She is known to have hypertension and obesity. She denies smoking, diabetes, hypercholesterolemia, or a family history of heart disease. She currently takes no medications. Physical examination is normal. The EKG shows nonspecific changes. Alprazolan was administred."
# test_sentence = 'Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. '
# test_sentence = 'The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.'
# test_sentence = 'Torsade de pointes ventricular tachycardia during  low dose intermittent dobutamine treatment in a patient with dilated cardiomyopathy and congestive heart failure.'
# test_sentence = 'A DNA molecule'
# test_sentence = '(a) Schematic drawing of the magnetic tweezers.'
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. I have arterial hypertension but I don’t take my medication every day.'
# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. It has started 2 hours ago, but its intensity isn’t decreasing. It started suddenly and I am also feeling pain in my back and my neck. The pain is continuous, and it did not worse when I am breathing. I have arterial hypertension but I don’t take my medication every day, I am sorry! I don’t smoke, and I don’t know If I have other diseases. My father has also arterial hypertension.'

# test_sentence = 'Doctor, I am feeling a very strong chest pain in the middle of my chest. It started suddenly and I am also feeling pain in my back and my neck. I have arterial hypertension but I don’t take my medication every day.'

test_sentence = """@NURSE
> Doctor, we have a man (51 years old) who entered the emergency department reporting chest pain. His vital signs are ABP: 144x92mmHG; HR: 78bpm; RR: 21rpm; Temp: 37oC; O2Sat: 98%.
@PATIENT
> Doctor, I am feeling chest pain since yesterday. The pain is continuous and is located just in the middle of my chest, worsening when I breathe and when I lay down on my bed. I suffer from arterial hypertension and smoke 20 cigarettes every day. My father had a "heart attack" at my age and I am very worried about it.
<b>PHYSICAL EXAMINATION</b> <br> The cardiac and pulmonary auscultation are normal; chest pain does not worse with palpation of the thorax; there is no jugular stasis nor lower limb edema.
Recently, we found that therapy with selegiline and L-dopa was associated with selective systolic orthostatic hypotension which was abolished by withdrawal of selegiline. 
The cells were examined in a Zeiss LSM 510 laser scanning microscope equipped with a Plan - Apochromate 63x/1.4 oil immersion objective.
 """



tokenized_sentence = tokenizer.encode(test_sentence)
# input_ids = torch.tensor([tokenized_sentence]).cuda()
input_ids = torch.tensor([tokenized_sentence])

with torch.no_grad():
    output = model(input_ids)
# print(output)
label_indices = np.argmax(output[0].to('cpu').numpy(), axis=2)
print(label_indices)
tokens = tokenizer.convert_ids_to_tokens(input_ids.to('cpu').numpy()[0])
new_tokens, new_labels = [], []
for token, label_idx in zip(tokens, label_indices[0]):
    if token.startswith("##"):
        new_tokens[-1] = new_tokens[-1] + token[2:]
    else:
#         print('else')
#         print(new_labels)
#         print(new_tokens)
        new_labels.append(labels[label_idx])
        new_tokens.append(token)
for token, label in zip(new_tokens, new_labels):
    print("{}\t\t{}".format(token, label))


[[6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 2 5 6 6 6 6 6 6 6 6 6 6 6
  6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6 6 6 6 6 6 6 2 5 6 6 6 6 2 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6 6 6 6 6 6 6 6 2 5 5 5 5 5 6 6 6 6 6 6 6 6 6 6 6 6 2 5 6 6 6 6 6 6
  6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6
  6 6 6 6 2 5 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 2 6 5 5 5 5 6 6 6 5 5 6 6 6 6
  6 6 6 6 1 1 4 4 6 1 4 4 4 6 6 6 6 2 6 6 6 5 5 5 5 5 5 5 5 5 6 6 6 6 6 6
  1 1 4 4 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 6 4 6 6 6 6 6 6 6
  6 6 6 6 6 6 6 6]]
[CLS]		O
@		O
NURSE		O
>		O
Doctor		O
,		O
we		O
have		O
a		O
man		O
(		O
51		O
years		O
old		O
)		O
who		O
entered		O
the		O
emergency		O
department		O
reporting		O
chest		B-Disease
pain		I-Disease
.		O
His		O
vital		O
signs		O
are		O
ABP		O
:		O
144x92mmHG		O
;		O
HR		O
:		O
78bpm		O
;		O
RR		O
:		O
21rpm		O
;		O
Temp		O
:		O
37oC		O
;		O
O2Sat		O
:		O
98		O
%		O
.		O
@		O
PATIENT		O